In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Conexion a la BD

In [2]:
def dataBase_connection():
    """This function connects with the data Warehouse and return a diccionary with the dataframes"""

    host = '127.0.0.1'  # Cambia esto por el endpoint de tu RDS
    port = 3307
    user = 'airflow'  # Cambia esto por tu usuario
    password = 'airflow'  # Cambia esto por tu contraseña
    database = 'airflow'  # Cambia esto por tu base de datos
    engine = None

    
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    

    # Create a list of the tables in the database
    tables =['temperature_WH','humidity_WH','wind_speed_WH','pressure_WH','weather_description_WH','wind_direction_WH']
   
    dataframes = {}
    # Loop through the tables and read them into pandas dataframes
    for table in tables:
        df = pd.read_sql_table(table, engine)
        dataframes[table] = df

    # Close the connection
    engine.dispose()
    return dataframes

In [ ]:
#Get the dataframes from the databases
dataframes = dataBase_connection()

In [ ]:
#List of the names of the tables in the databases
dataframes.keys()

dict_keys(['temperature_WH', 'humidity_WH', 'wind_speed_WH', 'pressure_WH', 'weather_description_WH', 'wind_direction_WH'])

## Temperature

In [ ]:

# Change the type of data int the columns datetime 
dataframes['temperature_WH']['datetime'] = pd.to_datetime(dataframes['temperature_WH']['datetime'])


In [ ]:
dataframes['temperature_WH']['datetime'].dt.year.unique()

In [8]:
def graphic(df,year,city):
    """This function creates a graphic with the temperature data for a specific year"""

    #Filter the dataframe to get only the data of 2023
    temperature_df= df[df['datetime'].dt.year == year].copy()

    #Get the average of the temperature by month
    temperature_df['month'] = temperature_df['datetime'].dt.month
    
    #Change the name of the month
    temperature_df['Month'] = temperature_df['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


    #Get the average of the temperature by month
    temperature_df = temperature_df.groupby(['Month']).mean().reset_index()

    # Show int ascending order
    temperature_df = temperature_df.sort_values(by='month', ascending=True)

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Bar( x = temperature_df['Month'], y = temperature_df[city], name = f'Temperature {year}', 
            marker= dict(color=temperature_df[city], colorscale = [[0,'rgb(255,200,200)'],[1,'rgb(255,0,0)']],# scale from red to white
                            colorbar=dict(title='Temp °F')))
   

    return fig



In [121]:
def linear_graphic(df,city):
    """This function creat a linear grophic where show the temperature avarage for each year in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df[city], mode='lines+markers', name=f'Temperature {city}',marker_color='red'))
    
    return fig


In [19]:
def growth_rate(df,city):
    """This function calculates  an graphs the growth rate of the temperature in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    #Calculate the growth rate
    df['growth_rate'] = df[city].pct_change() * 100

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df['growth_rate'], mode='lines+markers', name=f'Growth rate {city}'))
    

    return fig

### Los Angeles

#### Graphics where show the temperature by month from 2012 to 2017

In [82]:
# Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic(dataframes['temperature_WH'],2012,'Los Angeles'), row=1, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2013,'Los Angeles'), row=1, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2014,'Los Angeles'), row=2, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2015,'Los Angeles'), row=2, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2016,'Los Angeles'), row=3, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2017,'Los Angeles'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Temperatures in Los Angeles from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average temperature from 2012 to 2017

In [122]:
#Get the rsult of the function linear_graphic
fig = linear_graphic(dataframes['temperature_WH'],'Los Angeles')
fig.update_layout(
    title="Temperature in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Temperature °F",
    showlegend=True
)
fig.show()

### Graphic where show the growth rate 

In [90]:
fig = growth_rate(dataframes['temperature_WH'],'Los Angeles')
fig.update_layout(
    title="Growth rate of temperature in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

In [98]:
dataframes['temperature_WH'].columns

Index(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa',
       'Nahariyya', 'Jerusalem', 'id', 'year'],
      dtype='object')

### Houston

#### Graphics where show the temperature by month from 2012 to 2017

In [99]:
# Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic(dataframes['temperature_WH'],2012,'Houston'), row=1, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2013,'Houston'), row=1, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2014,'Houston'), row=2, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2015,'Houston'), row=2, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2016,'Houston'), row=3, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2017,'Houston'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Temperatures in Houston from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average temperature from 2012 to 2017

In [100]:
#Get the rsult of the function linear_graphic
fig = linear_graphic(dataframes['temperature_WH'],'Houston')
fig.update_layout(
    title="Temperature in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Temperature °F",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year of the temperature

In [101]:
fig = growth_rate(dataframes['temperature_WH'],'Houston')
fig.update_layout(
    title="Growth rate of temperature in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

### Chicago

#### Graphics where show the temperature by month from 2012 to 2017

In [102]:
# Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic(dataframes['temperature_WH'],2012,'Chicago'), row=1, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2013,'Chicago'), row=1, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2014,'Chicago'), row=2, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2015,'Chicago'), row=2, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2016,'Chicago'), row=3, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2017,'Chicago'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Temperatures in Chicago from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average temperature from 2012 to 2017

In [ ]:
#Get the rsult of the function linear_graphic
fig = linear_graphic(dataframes['temperature_WH'],'Chicago')
fig.update_layout(
    title="Temperature in Chicago from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Temperature °F",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year

In [104]:
fig = growth_rate(dataframes['temperature_WH'],'Chicago')
fig.update_layout(
    title="Growth rate of temperature in Chicago from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

### Conclusion

We can see how during the years the temperature is increasing in the main three cities with more agriculture production.The largest temperature increase was recorded from 2012 to 2013 and the temperature increase from 2012 to 2017 by 4 degrees on average

## Humidity

In [105]:
dataframes['humidity_WH']

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem,id
0,2012-10-01 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,1
1,2012-10-01 13:00:00,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0,2
2,2012-10-01 14:00:00,76.0,80.0,87.0,80.0,88.0,81.0,21.0,23.0,49.0,...,57.0,91.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0,3
3,2012-10-01 15:00:00,76.0,80.0,86.0,80.0,88.0,81.0,21.0,23.0,49.0,...,57.0,87.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0,4
4,2012-10-01 16:00:00,77.0,80.0,85.0,79.0,88.0,81.0,21.0,23.0,49.0,...,57.0,84.0,68.0,52.0,62.0,22.0,51.0,51.0,50.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45195,2017-11-27 15:00:00,0.0,93.0,0.0,87.0,87.0,87.0,29.0,36.0,38.0,...,0.0,73.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,45196
45196,2017-11-27 16:00:00,0.0,93.0,0.0,100.0,82.0,77.0,27.0,32.0,39.0,...,0.0,58.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,45197
45197,2017-11-27 17:00:00,0.0,100.0,0.0,87.0,87.0,100.0,28.0,31.0,34.0,...,0.0,62.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,45198
45198,2017-11-27 18:00:00,0.0,93.0,0.0,87.0,72.0,87.0,28.0,28.0,24.0,...,0.0,58.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,45199


In [107]:

# Change the type of data int the columns datetime 
dataframes['humidity_WH']['datetime'] = pd.to_datetime(dataframes['humidity_WH']['datetime'])


In [108]:
dataframes['humidity_WH']['datetime'].dt.year.unique()

array([2012, 2013, 2014, 2015, 2016, 2017])

In [111]:
def graphic_humidity(df,year,city):
    """This function creates a graphic with the Humidity data for a specific year"""

    #Filter the dataframe to get only the data of 2023
    humidity_df= df[df['datetime'].dt.year == year].copy()

    #Get the average of the temperature by month
    humidity_df['month'] = humidity_df['datetime'].dt.month
    
    #Change the name of the month
    humidity_df['Month'] = humidity_df['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


    #Get the average of the temperature by month
    humidity_df = humidity_df.groupby(['Month']).mean().reset_index()

    # Show int ascending order
    humidity_df = humidity_df.sort_values(by='month', ascending=True)

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Bar( x = humidity_df['Month'], y = humidity_df[city], name = f'Humidity {year}', 
            marker= dict(color=humidity_df[city], colorscale = [[0,'rgb(200,200,255)'],[1,'rgb(0,0,255)']],# scale from blue to white
                            colorbar=dict(title='Humidity %')))
   

    return fig



In [123]:
def linear_graphic_h(df,city):
    """This function creat a linear grophic where show the Humidity avarage for each year in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df[city], mode='lines+markers', name=f'Humidity {city}' ,marker_color='blue'))
    
    return fig


### Los Angeles

#### Graphics where show the Humidity by month from 2012 to 2017

In [119]:
# graphic_humidity of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2012,'Los Angeles'), row=1, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2013,'Los Angeles'), row=1, col=2)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2014,'Los Angeles'), row=2, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2015,'Los Angeles'), row=2, col=2)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2016,'Los Angeles'), row=3, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2017,'Los Angeles'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Humidity in Los Angeles from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average Humidity from 2012 to 2017

In [124]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_h(dataframes['humidity_WH'],'Los Angeles')
fig.update_layout(
    title="Humidity in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Humidity %",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year of the humidity

In [126]:
fig = growth_rate(dataframes['humidity_WH'],'Los Angeles')
fig.update_layout(
    title="Growth rate of humidity in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

### Houston

#### Graphics where show the Humidity by month from 2012 to 2017

In [127]:
# graphic_humidity of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2012,'Houston'), row=1, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2013,'Houston'), row=1, col=2)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2014,'Houston'), row=2, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2015,'Houston'), row=2, col=2)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2016,'Houston'), row=3, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2017,'Houston'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Humidity in Houston from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average Humidity from 2012 to 2017

In [128]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_h(dataframes['humidity_WH'],'Houston')
fig.update_layout(
    title="Humidity in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Humidity %",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year of the humidity

In [130]:
fig = growth_rate(dataframes['humidity_WH'],'Houston')
fig.update_layout(
    title="Growth rate of humidity in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

### Chicago

#### Graphics where show the Humidity by month from 2012 to 2017

In [131]:
# graphic_humidity of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2012,'Chicago'), row=1, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2013,'Chicago'), row=1, col=2)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2014,'Chicago'), row=2, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2015,'Chicago'), row=2, col=2)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2016,'Chicago'), row=3, col=1)
fig.add_trace(graphic_humidity(dataframes['humidity_WH'],2017,'Chicago'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Humidity in Chicago from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average Humidity from 2012 to 2017

In [132]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_h(dataframes['humidity_WH'],'Chicago')
fig.update_layout(
    title="Humidity in Chicago from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Humidity %",
    showlegend=True
)
fig.show()

## Wind Speed

In [133]:

# Change the type of data int the columns datetime 
dataframes['wind_speed_WH']['datetime'] = pd.to_datetime(dataframes['wind_speed_WH']['datetime'])


In [134]:
dataframes['wind_speed_WH']['datetime'].dt.year.unique()

array([2012, 2013, 2014, 2015, 2016, 2017])

In [135]:
dataframes['wind_speed_WH']

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem,id
0,2012-10-01 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1
1,2012-10-01 13:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,...,7.0,4.0,3.0,1.0,0.0,8.0,2.0,2.0,2.0,2
2,2012-10-01 14:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,...,7.0,4.0,3.0,3.0,0.0,8.0,2.0,2.0,2.0,3
3,2012-10-01 15:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,...,7.0,4.0,3.0,3.0,0.0,8.0,2.0,2.0,2.0,4
4,2012-10-01 16:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,...,7.0,4.0,3.0,3.0,0.0,8.0,2.0,2.0,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45195,2017-11-27 15:00:00,0.0,1.0,0.0,2.0,2.0,3.0,3.0,1.0,1.0,...,0.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,45196
45196,2017-11-27 16:00:00,0.0,3.0,0.0,4.0,4.0,1.0,2.0,1.0,1.0,...,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,45197
45197,2017-11-27 17:00:00,0.0,3.0,0.0,2.0,2.0,2.0,9.0,5.0,1.0,...,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,45198
45198,2017-11-27 18:00:00,0.0,3.0,0.0,1.0,2.0,1.0,10.0,4.0,1.0,...,0.0,5.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,45199


In [136]:
def graphic_wind(df,year,city):
    """This function creates a graphic with the temperature data for a specific year"""

    #Filter the dataframe to get only the data of 2023
    temperature_df= df[df['datetime'].dt.year == year].copy()

    #Get the average of the temperature by month
    temperature_df['month'] = temperature_df['datetime'].dt.month
    
    #Change the name of the month
    temperature_df['Month'] = temperature_df['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


    #Get the average of the temperature by month
    temperature_df = temperature_df.groupby(['Month']).mean().reset_index()

    # Show int ascending order
    temperature_df = temperature_df.sort_values(by='month', ascending=True)

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Bar( x = temperature_df['Month'], y = temperature_df[city], name = f'wind speed {year}', 
            marker= dict(color=temperature_df[city], colorscale = [[0,'rgb(200,255,200)'],[1,'rgb(0,255,0)']],# scale from red to white
                            colorbar=dict(title='speed m/s')))
   

    return fig



In [137]:
def linear_graphic_w(df,city):
    """This function creat a linear grophic where show the temperature avarage for each year in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df[city], mode='lines+markers', name=f'Speed wind {city}',marker_color='green'))
    
    return fig


## Los Angeles

#### Graphics where show the temperature by month from 2012 to 2017

In [138]:
# Graphic_graphic_wind of the wind_speed where the x axis are the months of a year and the y axis are the wind_speed in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2012,'Los Angeles'), row=1, col=1)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2013,'Los Angeles'), row=1, col=2)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2014,'Los Angeles'), row=2, col=1)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2015,'Los Angeles'), row=2, col=2)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2016,'Los Angeles'), row=3, col=1)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2017,'Los Angeles'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="wind_speeds in Los Angeles from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average wind speed from 2012 to 2017

In [140]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_w(dataframes['wind_speed_WH'],'Los Angeles')
fig.update_layout(
    title="Wind speed in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Wind speed m/s",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year of the wind speed

In [ ]:
fig = growth_rate(dataframes['wind_speed_WH'],'Los Angeles')
fig.update_layout(
    title="Growth rate of wind speed in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

## Houston

#### Graphics where show the wind speed by month from 2012 to 2017

In [142]:
# Graphic_graphic_wind of the wind_speed where the x axis are the months of a year and the y axis are the wind_speed in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2012,'Houston'), row=1, col=1)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2013,'Houston'), row=1, col=2)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2014,'Houston'), row=2, col=1)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2015,'Houston'), row=2, col=2)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2016,'Houston'), row=3, col=1)
fig.add_trace(graphic_wind(dataframes['wind_speed_WH'],2017,'Houston'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="wind_speeds in Houston from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average wind speed from 2012 to 2017

In [143]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_w(dataframes['wind_speed_WH'],'Houston')
fig.update_layout(
    title="Wind speed in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Wind speed m/s",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year of the wind speed

In [144]:
fig = growth_rate(dataframes['wind_speed_WH'],'Houston')
fig.update_layout(
    title="Growth rate of wind speed in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

## Pressure

In [6]:

# Change the type of data int the columns datetime 
dataframes['pressure_WH']['datetime'] = pd.to_datetime(dataframes['pressure_WH']['datetime'])


In [15]:
def graphic_pressure(df,year,city):
    """This function creates a graphic with the pressure data for a specific year"""

    #Filter the dataframe to get only the data of 2023
    pressure_df= df[df['datetime'].dt.year == year].copy()

    #Get the average of the temperature by month
    pressure_df['month'] = pressure_df['datetime'].dt.month
    
    #Change the name of the month
    pressure_df['Month'] = pressure_df['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


    #Get the average of the temperature by month
    pressure_df = pressure_df.groupby(['Month']).mean().reset_index()

    # Show int ascending order
    pressure_df = pressure_df.sort_values(by='month', ascending=True)

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Bar( x = pressure_df['Month'], y = pressure_df[city], name = f'Pressure {year}', 
            marker= dict(color=pressure_df[city], colorscale = [[0,'rgb(255,255,255)'],[1,'rgb(0,0,0)']],# scale from red to white
                            colorbar=dict(title='Pressure hPa')))
   

    return fig



In [10]:
def linear_graphic_pressure(df,city):
    """This function creat a linear grophic where show the pressure avarage for each year in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df[city], mode='lines+markers', name=f'Pressure {city}',marker_color='black'))
    
    return fig


## Los Angeles

#### Graphics where show the temperature by month from 2012 to 2017

In [16]:
# graphic_pressure of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2012,'Los Angeles'), row=1, col=1)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2013,'Los Angeles'), row=1, col=2)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2014,'Los Angeles'), row=2, col=1)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2015,'Los Angeles'), row=2, col=2)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2016,'Los Angeles'), row=3, col=1)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2017,'Los Angeles'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Pressure in Los Angeles from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average pressure from 2012 to 2017

In [17]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_pressure(dataframes['pressure_WH'],'Los Angeles')
fig.update_layout(
    title="Pressure  Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Pressure ",
    showlegend=True
)
fig.show()

### Graphic where show the growth rate of the pressure

In [20]:
fig = growth_rate(dataframes['pressure_WH'],'Los Angeles')
fig.update_layout(
    title="Growth rate of temperature in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

## Houston

#### Graphics where show the pressure by month from 2012 to 2017

In [21]:
# graphic_pressure of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2012,'Houston'), row=1, col=1)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2013,'Houston'), row=1, col=2)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2014,'Houston'), row=2, col=1)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2015,'Houston'), row=2, col=2)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2016,'Houston'), row=3, col=1)
fig.add_trace(graphic_pressure(dataframes['pressure_WH'],2017,'Houston'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Pressure in Houston from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average pressure from 2012 to 2017

In [22]:
#Get the rsult of the function linear_graphic
fig = linear_graphic_pressure(dataframes['pressure_WH'],'Houston')
fig.update_layout(
    title="Pressure  Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Pressure ",
    showlegend=True
)
fig.show()

### Graphic where show the growth rate of the pressure

In [23]:
fig = growth_rate(dataframes['pressure_WH'],'Houston')
fig.update_layout(
    title="Growth rate of pressure in Houston from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()